# Lab | Hypothesis Testing

**Objective**

Welcome to the Hypothesis Testing Lab, where we embark on an enlightening journey through the realm of statistical decision-making! In this laboratory, we delve into various scenarios, applying the powerful tools of hypothesis testing to scrutinize and interpret data.

From testing the mean of a single sample (One Sample T-Test), to investigating differences between independent groups (Two Sample T-Test), and exploring relationships within dependent samples (Paired Sample T-Test), our exploration knows no bounds. Furthermore, we'll venture into the realm of Analysis of Variance (ANOVA), unraveling the complexities of comparing means across multiple groups.

So, grab your statistical tools, prepare your hypotheses, and let's embark on this fascinating journey of exploration and discovery in the world of hypothesis testing!

**Challenge 1**

In this challenge, we will be working with pokemon data. The data can be found here:

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/pokemon.csv

In [13]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind


In [14]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/pokemon.csv")
df.head()


,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


- We posit that Pokemons of type Dragon have, on average, more HP stats than Grass. Choose the propper test and, with 5% significance, comment your findings.

In [15]:
dragon_hp = df[df['Type 1'] == 'Dragon']['HP']
grass_hp = df[df['Type 1'] == 'Grass']['HP']

stat, pval = ttest_ind(dragon_hp, grass_hp, equal_var=False)

print(f"Dragon mean HP: {dragon_hp.mean():.1f} (n={len(dragon_hp)})")
print(f"Grass mean HP: {grass_hp.mean():.1f} (n={len(grass_hp)})")
print(f"t = {stat:.2f}, p-value = {pval:.4f}")


Dragon mean HP: 83.3 (n=32)
Grass mean HP: 67.3 (n=70)
t = 3.33, p-value = 0.0016


- We posit that Legendary Pokemons have different stats (HP, Attack, Defense, Sp.Atk, Sp.Def, Speed) when comparing with Non-Legendary. Choose the propper test and, with 5% significance, comment your findings.


In [16]:
legendary = df[df['Legendary']]
non_legendary = df[~df['Legendary']]
columns = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']

rows = []
for col in columns:
    stat, pval = ttest_ind(legendary[col], non_legendary[col], equal_var=False)
    rows.append({
        'stat': col,
        'legendary_mean': round(legendary[col].mean(), 1),
        'non_legendary_mean': round(non_legendary[col].mean(), 1),
        'p_value': pval
    })

pd.DataFrame(rows)


,stat,legendary_mean,non_legendary_mean,p_value
0,HP,92.7,67.2,1.002691e-13
1,Attack,116.7,75.7,2.520372e-16
2,Defense,99.7,71.6,4.826998e-11
3,Sp. Atk,122.2,68.5,1.551461e-21
4,Sp. Def,105.9,68.9,2.294933e-15
5,Speed,100.2,65.5,1.049016e-18


**Challenge 2**

In this challenge, we will be working with california-housing data. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/california_housing.csv

In [17]:
housing = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/california_housing.csv')
housing.head()


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


**We posit that houses close to either a school or a hospital are more expensive.**

- School coordinates (-118, 34)
- Hospital coordinates (-122, 37)

We consider a house (neighborhood) to be close to a school or hospital if the distance is lower than 0.50.

Hint:
- Write a function to calculate euclidean distance from each house (neighborhood) to the school and to the hospital.
- Divide your dataset into houses close and far from either a hospital or school.
- Choose the propper test and, with 5% significance, comment your findings.
 

In [18]:
school = (-118, 34)
hospital = (-122, 37)

coord = housing[['longitude', 'latitude']].to_numpy()

def distance(point, ref):
    return np.sqrt((point[:, 0] - ref[0])**2 + (point[:, 1] - ref[1])**2)

housing['close_school'] = distance(coord, school) <= 2
housing['close_hospital'] = distance(coord, hospital) <= 2
housing['close_any'] = housing['close_school'] | housing['close_hospital']

housing['close_any'].value_counts()


close_any
True     14895
False     2105
Name: count, dtype: int64

In [20]:
close_values = housing[housing['close_any']]['median_house_value']
far_values = housing[~housing['close_any']]['median_house_value']

stat, pval = ttest_ind(close_values, far_values, equal_var=False)

print(f"Mean price close: {close_values.mean():.0f} (n={len(close_values)})")
print(f"Mean price far: {far_values.mean():.0f} (n={len(far_values)})")
print(f"t = {stat:.2f}, p-value = {pval:.4f}")


Mean price close: 221832 (n=14895)
Mean price far: 104478 (n=2105)
t = 75.29, p-value = 0.0000
